In [ ]:
import cv2
import random
import os, glob
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
import pandas as pd
import csv
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from numpy import expand_dims
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam,Adamax,RMSprop
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet101, Xception,VGG19,InceptionResNetV2,ResNet50V2,InceptionV3,MobileNetV2,MobileNet
from tensorflow.keras.applications.resnet_v2 import preprocess_input, decode_predictions
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import Input, Dense, Flatten, AveragePooling2D, Dropout, BatchNormalization, Conv2D, MaxPooling2D,Activation,add

In [ ]:
LR = 0.001
epochs = 10
batch_size = 32

img_shape = (224,224,3)


In [ ]:
test_dir = "/kaggle/input/leukemia-classification/C-NMC_Leukemia/testing_data/C-NMC_test_final_phase_data/"
train01_hem_dir = "/kaggle/input/leukemia-classification/C-NMC_Leukemia/training_data/fold_0/hem/"
train02_hem_dir = "/kaggle/input/leukemia-classification/C-NMC_Leukemia/training_data/fold_1/hem/"
train03_hem_dir = "/kaggle/input/leukemia-classification/C-NMC_Leukemia/training_data/fold_2/hem/"
train01_all_dir = "/kaggle/input/leukemia-classification/C-NMC_Leukemia/training_data/fold_0/all/"
train02_all_dir = "/kaggle/input/leukemia-classification/C-NMC_Leukemia/training_data/fold_1/all/"
train03_all_dir = "/kaggle/input/leukemia-classification/C-NMC_Leukemia/training_data/fold_2/all/"
valid_dir = "/kaggle/input/leukemia-classification/C-NMC_Leukemia/validation_data/C-NMC_test_prelim_phase_data"

In [ ]:
data = []
labels = []
img_list1 = glob.glob(train01_hem_dir+'/*')
img_list2 = glob.glob(train02_hem_dir+'/*')
img_list3 = glob.glob(train03_hem_dir+'/*')
for img in img_list1[:]:
  image = cv2.imread(img)
  image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
  image = cv2.resize(image,(224,224))
  data.append(image)
  labels.append(1)
for img in img_list2[:]:
  image = cv2.imread(img)
  image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
  image = cv2.resize(image,(224,224))
  data.append(image)
  labels.append(1)
for img in img_list3[:]:
  image = cv2.imread(img)
  image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
  image = cv2.resize(image,(224,224))
  data.append(image)
  labels.append(1)
img_list4 = glob.glob(train01_all_dir+'/*')
img_list5 = glob.glob(train02_all_dir+'/*')
img_list6 = glob.glob(train03_all_dir+'/*')
for img in img_list4:
  image = cv2.imread(img)
  image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
  image = cv2.resize(image,(224,224))
  data.append(image)
  labels.append(0)
for img in img_list5:
  image = cv2.imread(img)
  image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
  image = cv2.resize(image,(224,224))
  data.append(image)
  labels.append(0)
for img in img_list6:
  image = cv2.imread(img)
  image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
  image = cv2.resize(image,(224,224))
  data.append(image)
  labels.append(0)


In [ ]:
print(len(data),len(labels))

In [ ]:
data = np.array(data)
labels = np.array(labels)

In [ ]:
data_valid = []
file_path = '/kaggle/input/leukemia-classification/C-NMC_Leukemia/validation_data/C-NMC_test_prelim_phase_data_labels.csv'
column_name = 'labels'
label_valid = []
valid_imglist = glob.glob(valid_dir+'/*')
for img in valid_imglist[:]:
  image = cv2.imread(img)
  image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
  image = cv2.resize(image,(224,224))
  data_valid.append(image)

with open(file_path, 'r', newline='') as csvfile:
    csvreader = csv.reader(csvfile)
    header = next(csvreader)
    column_index = header.index(column_name)
    for row in csvreader:
        label_valid.append(row[column_index])

# Now, 'column_data' contains the data from the specified column as a list
print(len(data_valid))
print(len(label_valid))
print(label_valid[0:5])

In [ ]:
data_valid = np.array(data_valid)
labels_valid = np.array(label_valid)

In [ ]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)
#labels_valid = lb.fit_transform(labels_valid)
#labels_valid = to_categorical(labels_valid)
(x_train, x_test, y_train, y_test) = train_test_split(
    data,
    labels,
    test_size=0.20,
    stratify=labels,
    random_state=42
)



print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
y_test

In [ ]:
def display_history(history_):
    fig, ax = plt.subplots(1,2, figsize=(12, 3))
    ax[0].plot(history_.history['loss'], color='b', label="training_loss")
    ax[0].plot(history_.history['val_loss'], color='r', label="validation_loss",axes =ax[0])
    ax[0].set_xlabel("Epoch")
    ax[0].set_ylabel("Loss")
    legend = ax[0].legend(loc='best', shadow=True)
    ax[1].plot(history_.history['accuracy'], color='b', label="training_accuracy")
    ax[1].plot(history_.history['val_accuracy'], color='r',label="validation_accuracy")
    ax[1].set_xlabel("Epoch")
    ax[1].set_ylabel("Accuracy")
    legend = ax[1].legend(loc='best', shadow=True)

def plot_metrices(model_):
    plt.figure()
    ax = plt.subplot()
    ax.set_title('Confusion Matrix')
    pred = model_.predict(x_test, batch_size = batch_size)
    pred = np.argmax(pred, axis = 1)
    cm = confusion_matrix(y_test.argmax(axis = 1), pred)
    classes=['normal', 'lukemia']
    sns.heatmap(cm, annot = True, xticklabels = classes, yticklabels = classes, cmap = 'Reds')

    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show

    print(classification_report(y_test.argmax(axis = 1), pred))
    total = sum(sum(cm))
    acc = (cm[0, 0] + cm[1, 1]) / total
    sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
    specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])

    print("ACC: {:.4f}".format(acc))
    print("Sensitivity: {:.4f}".format(sensitivity))
    print("Specificity: {:.4f}".format(specificity))

In [ ]:
model_name='EfficientNetB3'
base_model=tf.keras.applications.efficientnet.EfficientNetB3(include_top=False, weights="imagenet",input_shape=img_shape, pooling='max') 
x=base_model.output
x=keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 )(x)
x = Dense(256,activation='relu')(x)
x=Dropout(rate=.45, seed=123)(x)        
output=Dense(2, activation='softmax')(x)
model=Model(inputs=base_model.input, outputs=output)
model.compile(Adamax(learning_rate=.001), loss='categorical_crossentropy', metrics=['accuracy']) 

In [ ]:
model.summary()

In [ ]:
import tensorflow as tf

# Disable Grappler optimizer
tf.config.optimizer.set_experimental_options({"layout_optimizer": False})

In [ ]:
filepath = './best_weights_EFNB3_leukemia.hdf5'

earlystopping = EarlyStopping(monitor = 'val_accuracy', 
                              mode = 'max' , 
                              patience = 15,
                              verbose = 1)

checkpoint    = ModelCheckpoint(filepath, 
                                monitor = 'val_accuracy', 
                                mode='max', 
                                save_best_only=True, 
                                verbose = 1)

learning_rate = ReduceLROnPlateau(monitor = 'val_accuracy',
                                  mode = 'max',
                                  patience = 5,
                                  factor = 0.3,
                                  min_delta = 0.00001)


callback_list = [earlystopping, checkpoint, learning_rate]

In [ ]:
test_datagen = ImageDataGenerator()
train_datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.2, 1.2]
)

train_generator = train_datagen.flow(x_train, y_train, batch_size=batch_size, seed=27, shuffle=False)
val_generator = test_datagen.flow(x_test, y_test, batch_size=batch_size, shuffle=False)

In [ ]:
history = model.fit(train_generator,
                    epochs=epochs,
                    steps_per_epoch=len(x_train) // batch_size,
                    validation_data = val_generator,
                    validation_steps = len(x_test) // batch_size,
                    verbose=1,
                    callbacks = callback_list)

In [ ]:
#from tensorflow.keras.models import load_model

# Save the model
#model.save('efficientnetB3_leukemia_model.h5')

In [ ]:
# Load the model
#model = load_model('efficientnetB3_leukemia_model.h5')

In [ ]:
display_history(history)
plot_metrices(model)

In [ ]:
from tensorflow.keras.models import load_model
model1 = load_model('/kaggle/input/weight/best_weights_EFNB3_leukemia.hdf5')

In [ ]:
idx2 = random.randint(0, len(labels_valid))
img = data_valid[idx2]
plt.imshow(img)
img_resized = cv2.resize(img,(224,224))
input_img = np.expand_dims(img_resized,axis = 0)
print(idx2)
print(input_img.shape)

In [ ]:
pred = model1.predict(input_img)
print(pred)


In [ ]:
lu_val = pred[0][0]
nor_val = pred[0][1]

In [ ]:
if(lu_val>nor_val):
  print("Leukemia")
if(nor_val>lu_val):
  print("Normal")

In [ ]:
output_real = labels_valid[idx2]
print(output_real)
if(output_real=='1'):
  print("Leukemia")
if(output_real=='0'):
  print("Normal")


In [ ]:
import time
start = time.time()
final_pred = []
column_name = 'labels'
for img in data_valid[:300]:
    img_resized = cv2.resize(img,(224,224))
    input_img = np.expand_dims(img_resized,axis = 0)
    pred = model1.predict(input_img)
    print(pred)
    lu_val = pred[0][0]
    nor_val = pred[0][1]
    if(lu_val>nor_val):
        final_pred.append("1")
    if(nor_val>lu_val):
        final_pred.append("0")
end = time.time()
elapsed_time = end-start
print("Time taken:",elapsed_time)
print(final_pred)

In [ ]:
!pip install mpi4py